In [15]:
import os

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

from scipy.stats import norm

import matplotlib.pyplot as plt

In [16]:
downweight_granularity=['dtime','Direction','dday','month']

# Data Import

In [17]:
Data=pd.read_csv('R_Output_Test_Pax.csv',sep=',')
Data=Data[['DepDate','yday','dtime','Direction','month','dday','Dprio','NumPax']]

Map_Dates=DataFrame([Data['DepDate'].unique().tolist(),[pd.to_datetime(x) for x in Data['DepDate'].unique().tolist()]]).transpose()
Map_Dates.columns=['DepDate','DepDate_dt']
Map_Dates['DepDate']=Map_Dates['DepDate'].astype('str')

Data=Data.merge(Map_Dates,on=['DepDate'])
Data['DepDate']=Data['DepDate_dt']
Data=Data[[x for x in Data.columns if x!='DepDate_dt']]

Data['TicketDate']=Data['DepDate']-Data['Dprio'].apply(lambda x: datetime.timedelta(x))

Data=Data.loc[Data['TicketDate']<=pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d')),:]

Data=Data[[x for x in Data.columns if x!='TicketDate']]

Data['Dprio']=-Data['Dprio']
Data=Data.set_index(['DepDate','yday','dtime','Direction','month','dday','Dprio'])
Data=Data.groupby(level=[0,1,2,3,4,5,6]).sum().groupby(level=[0,1,2,3,4,5])['NumPax'].cumsum().reset_index()

Data['Dprio']=-Data['Dprio']

In [18]:
IdealCurve=pd.read_csv('IdealCurve.csv')

In [19]:
Data=Data.merge(IdealCurve,on=downweight_granularity+['Dprio'])

In [20]:
Data=Data[['DepDate','dtime','Direction','dday','Dprio','NumPax','Ideal_lower','Ideal_average_downweighted','Ideal_average_full','Ideal_upper']]\
.sort_values(by=['DepDate','Direction','dtime','Dprio'])

In [21]:
Data['NumPax']=Data['NumPax'].apply(lambda x: int(round(x)))
Data['Ideal_lower']=Data['Ideal_lower'].apply(lambda x: int(round(x)))
Data['Ideal_average_downweighted']=Data['Ideal_average_downweighted'].apply(lambda x: int(round(x)))
Data['Ideal_average_full']=Data['Ideal_average_full'].apply(lambda x: int(round(x)))
Data['Ideal_upper']=Data['Ideal_upper'].apply(lambda x: int(round(x)))

In [22]:
deptime=[str(x/60).split('.')[0]+':'+str(round(float('0.'+str(x/60).split('.')[1])*60)) for x in Data['dtime'].unique()]
deptime=[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]
deptime=['0'+x if len(x.split(':')[0])==1 else x for x in deptime]

Map_DepTime=DataFrame([Data['dtime'].unique(),[x+'0' if len(x.split(':')[1])==1 else x for x in deptime]]).transpose()
Map_DepTime.columns=['dtime','deptime']
Map_DepTime['dtime']=Map_DepTime['dtime'].astype('int')

Data=Data.merge(Map_DepTime,on='dtime')

Data['dtime']=Data['deptime']
Data=Data[[x for x in Data.columns if x!='deptime']]

In [23]:
Data=Data.sort_values(by=['DepDate','dtime'])
Data=Data.rename(columns={'NumPax': 'Actual Bookings', 'Ideal_lower': 'Ramp-up frontier',
'Ideal_average_downweighted': 'Ideal curve (80% LF)', 'Ideal_average_full': 'Ideal curve (100% LF)',
'Ideal_upper': 'Phase-down frontier'})

In [24]:
Data.to_csv('GUI_in.csv',index=False)

In [25]:
Data.loc[Data['DepDate']=='2019-04-11',:]

,DepDate,dtime,Direction,dday,Dprio,Actual Bookings,Ramp-up frontier,Ideal curve (80% LF),Ideal curve (100% LF),Phase-down frontier
55807,2019-04-11,06:45,O,Thursday,1,48,52,56,70,70
55808,2019-04-11,06:45,O,Thursday,2,49,50,55,68,69
55809,2019-04-11,06:45,O,Thursday,3,50,48,53,67,68
55810,2019-04-11,06:45,O,Thursday,4,50,47,52,65,67
55811,2019-04-11,06:45,O,Thursday,5,50,45,51,63,66
55812,2019-04-11,06:45,O,Thursday,6,49,43,49,62,65
55813,2019-04-11,06:45,O,Thursday,7,44,42,48,60,64
55814,2019-04-11,06:45,O,Thursday,8,43,40,47,59,63
55815,2019-04-11,06:45,O,Thursday,9,40,39,46,57,62
55816,2019-04-11,06:45,O,Thursday,10,40,38,45,56,61
